## Generating and Visualizing Antipodal Grasps on Book

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import trimesh
import sys
from typing import List 
from IPython.display import clear_output
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Concatenate,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    PointCloud,
    RigidTransform,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
    Context,
    Diagram,
    PointCloud,
    Simulator,
    TrajectorySource,
    Solve,
    RotationMatrix,
    MultibodyPlant,
    eq,
    StateInterpolatorWithDiscreteDerivative,
    MinimumDistanceLowerBoundConstraint,
    RollPitchYaw,
    SolutionResult,
    CollisionFilterDeclaration,
    GeometrySet,
    Role,
)
from pydrake.geometry import Meshcat
from pydrake.multibody import inverse_kinematics

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation import running_as_notebook
from manipulation.scenarios import AddFloatingRpyJoint, AddRgbdSensors, ycb, AddMultibodyTriad, MakeManipulationStation
from manipulation.utils import ConfigureParser
from manipulation.clutter import GraspCandidateCost, GenerateAntipodalGraspCandidate
from manipulation.icp import IterativeClosestPoint
from manipulation.pick import (
    MakeGripperCommandTrajectory,
    MakeGripperFrames,
    MakeGripperPoseTrajectory,
)
# Own utils
from hwstation.utils import init_diagram, fix_input_port, visualize_diagram, filterCollsionGeometry, get_iiwa_joint_state
from hwstation.add_objects import *
from perception.icp import run_table_multi_book_icp, check_quality_of_icp
from rrt.rrt_planning import *
from grasp.grasping import *

from enum import Enum
import pandas as pd

from manipulation.station import MakeHardwareStation, load_scenario
from manipulation.station import AddPointClouds
from random import random
from manipulation.pick import *
#from manipulation.pick import *

import matplotlib.pyplot as plt

## 1. Setup environment

In [3]:
# Start meshcat
try:
    meshcat = Meshcat(7005)
except:
    pass #This error appears if this cell is executed twice (port 7006 is already taken then)

scenario_data = get_library_scenario_data()
diagram_plan, visualizer, simulator = init_diagram(meshcat, scenario_data)
fix_input_port(diagram_plan, simulator)
diagram_plan_context = diagram_plan.CreateDefaultContext()
scene_graph = diagram_plan.GetSubsystemByName("station").GetSubsystemByName("scene_graph")
sg_context = scene_graph.GetMyContextFromRoot(diagram_plan_context)
filterCollsionGeometry(scene_graph, sg_context)
q0_init = get_iiwa_joint_state(diagram_plan, diagram_plan_context)
simulator.AdvanceTo(1.0)
diagram_plan_context = simulator.get_mutable_context()


INFO:drake:Meshcat listening for connections at http://localhost:7005


Station input port size:  <pydrake.systems.framework.InputPort object at 0x7fff89dde830>
plant.GetStateNames():  154
plant.GetActuatorNames():  12
Fixing input port of size:  20
Fixing input port of size:  1


==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html



## 2. Running ICP to find point cloud

In [4]:
from perception.icp import *
from hwstation.utils import visualize_frame

In [5]:
icp_cloud_list, X_MS_hat_list = run_table_multi_book_icp(diagram_plan, diagram_plan_context, meshcat,icp_limit=0)
for idx, X_MS_hat in enumerate(X_MS_hat_list):
    visualize_frame(str(idx),meshcat, X_MS_hat)

fixing improper rotation
